In [ ]:
CREATE OR REPLACE TABLE DATA_GLD.SBN_CALCULATED_EVENTS (
  event_url        VARCHAR,
  event_time       TIMESTAMP_NTZ,
  event_name       VARCHAR,
  group_name       VARCHAR,
  city_name        VARCHAR,
  category_name    VARCHAR,
  yes_rsvp_count   NUMBER(38,0),
  rsvp_limit       NUMBER(38,0),
  fill_rate_pct    NUMBER(38,2),
  load_ts          TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
  CONSTRAINT uq_mart_events_min UNIQUE (event_url, event_time)
);

In [ ]:
CREATE OR REPLACE TABLE MEETUP_DB.DATA_GLD.SBN_CALCULATED_EVENTS{{ ts_nodash }} AS
SELECT
  NULLIF(TRIM(e.event_url), '')                           AS event_url,
  e.event_time,
  NULLIF(TRIM(e.event_name), '')                          AS event_name,
  NULLIF(TRIM(g.group_name), '')                          AS group_name,
  COALESCE(NULLIF(TRIM(vc.city), ''), NULLIF(TRIM(gc.city), ''))         AS city_name,
  NULLIF(TRIM(c.name_category), '')                       AS category_name,
  e.yes_rsvp_count,
  e.rsvp_limit,
  CASE WHEN e.rsvp_limit > 0
       THEN ROUND(100.0 * e.yes_rsvp_count / e.rsvp_limit, 2) END       AS fill_rate_pct,
  CURRENT_TIMESTAMP()                                                   AS load_ts
FROM MEETUP_DB.DATA_SLV.FACT_EVENTS AS e
LEFT JOIN MEETUP_DB.DATA_SLV.DIM_GROUPS     g  ON g.id_group    = e.id_group
LEFT JOIN MEETUP_DB.DATA_SLV.DIM_CATEGORIES c  ON c.id_category = g.id_category
LEFT JOIN MEETUP_DB.DATA_SLV.DIM_CITIES     gc ON gc.id_city    = g.id_city
LEFT JOIN MEETUP_DB.DATA_SLV.DIM_VENUES     v  ON v.venue_id    = e.id_venue
LEFT JOIN MEETUP_DB.DATA_SLV.DIM_CITIES     vc ON vc.id_city    = v.id_city
WHERE e.event_url IS NOT NULL;
